# Grammar Correction using LLM

In [1]:
import torch
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Dataset

In [2]:
# Grammar Correction Dataset : https://www.kaggle.com/datasets/satishgunjal/grammar-correction

data = pd.read_csv("./dataset/Grammar Correction.csv", sep=",")
data.head()

,Serial Number,Error Type,Ungrammatical Statement,Standard English
0,1,Verb Tense Errors,I goes to the store everyday.,I go to the store everyday.
1,2,Verb Tense Errors,They was playing soccer last night.,They were playing soccer last night.
2,3,Verb Tense Errors,She have completed her homework.,She has completed her homework.
3,4,Verb Tense Errors,He don't know the answer.,He doesn't know the answer.
4,5,Verb Tense Errors,The sun rise in the east.,The sun rises in the east.


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018 entries, 0 to 2017
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Serial Number            2018 non-null   int64 
 1   Error Type               2018 non-null   object
 2   Ungrammatical Statement  2018 non-null   object
 3   Standard English         2018 non-null   object
dtypes: int64(1), object(3)
memory usage: 63.2+ KB


In [4]:
# count of error types

error_counts = data['Error Type'].value_counts()
print(error_counts)

Error Type
Sentence Structure Errors                         103
Verb Tense Errors                                 100
Subject-Verb Agreement                            100
Article Usage                                     100
Spelling Mistakes                                 100
Preposition Usage                                  95
Punctuation Errors                                 60
Relative Clause Errors                             51
Gerund and Participle Errors                       50
Abbreviation Errors                                50
Slang, Jargon, and Colloquialisms                  50
Negation Errors                                    50
Incorrect Auxiliaries                              50
Ambiguity                                          50
Tautology                                          50
Lack of Parallelism in Lists or Series             50
Mixed Metaphors/Idioms                             50
Parallelism Errors                                 49
Contractions Erro

## Model: T5

In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [6]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

## Train Test Split

In [7]:
from sklearn.model_selection import train_test_split

# 80 : 20
train_data, temp_data = train_test_split(data, test_size=0.2, random_state=42)

# 10 : 10
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")
print(f"Test set size: {len(test_data)}")

Training set size: 1614
Validation set size: 202
Test set size: 202


## Tokenize data

In [8]:
def tokenize_data(data, tokenizer, max_length=128):
    inputs = tokenizer(
        data['Ungrammatical Statement'].tolist(),
        max_length=max_length,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    outputs = tokenizer(
        data['Standard English'].tolist(),
        max_length=max_length,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    return inputs, outputs

# tokenmize datasets
train_inputs, train_outputs = tokenize_data(train_data[['Ungrammatical Statement', 'Standard English']], tokenizer)
val_inputs, val_outputs = tokenize_data(val_data[['Ungrammatical Statement', 'Standard English']], tokenizer)
test_inputs, test_outputs = tokenize_data(test_data[['Ungrammatical Statement', 'Standard English']], tokenizer)

train_inputs = {key: value.to(device) for key, value in train_inputs.items()}
train_outputs = train_outputs['input_ids'].to(device)

val_inputs = {key: value.to(device) for key, value in val_inputs.items()}
val_outputs = val_outputs['input_ids'].to(device)

test_inputs = {key: value.to(device) for key, value in test_inputs.items()}
test_outputs = test_outputs['input_ids'].to(device)

In [9]:
train_inputs

{'input_ids': tensor([[  451, 15687,   160,  ...,     0,     0,     0],
         [  216,   228,    43,  ...,     0,     0,     0],
         [   27,    43,     3,  ...,     0,     0,     0],
         ...,
         [ 1902,     5, 12587,  ...,     0,     0,     0],
         [   37,  1595,   562,  ...,     0,     0,     0],
         [   37,   167,   167,  ...,     0,     0,     0]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}

## Data Loader

In [10]:
from torch.utils.data import DataLoader, TensorDataset

# tensordatasets
train_dataset = TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], train_outputs)
val_dataset = TensorDataset(val_inputs['input_ids'], val_inputs['attention_mask'], val_outputs)
test_dataset = TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'], test_outputs)

# dataloaders
train_loader = DataLoader(train_dataset, batch_size=8)
val_loader = DataLoader(val_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

In [11]:
train_loader

## Fine-Tune T5 with Error Type

We use the Standard English as the target output and input the ungrammatical sentence with the error type prepended so that the model may also identify type of error

In [12]:
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# fine-tuning loop
model.train()

epochs = 3

for epoch in tqdm(range(epochs), desc='Finetuning..'):
    print(f"Epoch {epoch + 1}")

    # training
    model.train()
    for batch in tqdm(train_loader, desc='Training..'):
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        print(f"Training Loss: {loss.item()}")

    # validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validating..'):
            input_ids, attention_mask, labels = batch
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            val_loss += outputs.loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Validation Loss: {avg_val_loss}")

Finetuning..:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1



Training..:   0%|          | 1/202 [00:01<06:23,  1.91s/it]

Training Loss: 16.158187866210938



Training..:   1%|          | 2/202 [00:02<03:31,  1.06s/it]

...

Training Loss: 0.07712844759225845



Training..: 100%|██████████| 202/202 [01:43<00:00,  1.95it/s]


Training Loss: 0.03663608431816101



Finetuning..: 100%|██████████| 3/3 [05:17<00:00, 105.75s/it]

Validation Loss: 0.030435680161015347


In [13]:
# evaluate on test set

model.eval()
test_loss = 0
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        test_loss += outputs.loss.item()

avg_test_loss = test_loss / len(test_loader)
print(f"Test Loss: {avg_test_loss}")

Test Loss: 0.025267765391618013
